## Correlation Power Analysis attack on AES-128

In [1]:
%matplotlib inline

Here we will declare few basic functions and constants

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import Crypto.Cipher.AES as AES

textIn = np.load(r'task1/textin.npy')
textOut = np.load(r'task1/textout.npy')
traces = np.load(r'task1/traces.npy')

# AES-128 substitution box

Sbox = (
0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76,
0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0,
0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15,
0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75,
0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84,
0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf,
0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8,
0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2,
0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73,
0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb,
0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79,
0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08,
0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a,
0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e,
0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf,
0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16)


Here is correlation implementation (but we will use built in numpy version for performance purposes)

In [3]:
def cor2(x, y):
    meanX = np.mean(x)
    meanY = np.mean(y)
    cov = sum([(x[i] - meanX)*(y[i] - meanY) for i in range(len(x))])
    meanQuadraticX = sum([(x[i] - meanX)**2 for i in range(len(x))]) ** 0.5
    meanQuadraticY = sum([(y[i] - meanY)**2 for i in range(len(y))]) ** 0.5
    res = cov / (meanQuadraticX * meanQuadraticY)
    return res

In [4]:
def cor(x, y):
    return np.corrcoef(x, y)[1, 0]

Hamming weight implementation

In [5]:
def hamming(x):
    b = bin(x)[2:]
    res = b.count('1')
    return res

Next, we will make a prediction function, which purpose is to collect data from each input sample data into one set


In [6]:
def getPrediction(textIn, byteNum, byteVal):
    prediction = np.zeros(len(textIn))
    for i in range(len(textIn)):
        prediction[i] = hamming(Sbox[textIn[i][byteNum] ^ byteVal])
    return prediction

And now, the main function. We will make result key array, which will be built from 16 8-bit values (128 bit key), each of which we will test on all measures, taken troughout traces (5000 measures taken in each trace, simmilary to the way we built our prediction array, we will try each measure **_i_** and from each trace we take **_ith_** value and add it to **_sample_** array), and will gather one which correlate the most with our predicted data. After we tested all 255 values we will keep the one with highest correlation coefficient and proceed to the nect byte of our key

In [7]:
def guessKey(textIn, traces):
    key = np.zeros(16)    # result key
    for byte_num in range(16):
        tmpRes = [0 for i in range(256)]    # temporary result
        for byte_val in range(256):
            mxVal = 0   # here we will store maximum value of correlation for a certain byte value
            mxIndx = None
            prediction = getPrediction(textIn, byte_num, byte_val)   # our prediction built on byte value and input data
            for i, sample in enumerate(zip(*traces)):
                correlation = cor(prediction, sample)
                if abs(correlation) > mxVal:  
                    mxVal = abs(correlation)
                    mxIndx = i
            tmpRes[byte_val] = (mxVal, mxIndx)
        b = max(enumerate(tmpRes), key=lambda x: x[1][0])    # max correlation throughout all byte value assumptions
        msg = 'byte number: {}\nbyte value:{}\ncorrelation value:{}\nleakage place:{}\n\n\n'
        print(msg.format(byte_num, b[0], b[1][0], b[1][1]))
        key[byte_num] = b[0]
    return key

On my laptop (Win 10, core i3 6006u) it took about 50 minutes to gather all results and reveal the key. Here I'd like to skip this process and insert the key manually.

In [17]:
key = b'N0_GuT5_No_S70RY'

And now One would like to take a look at some test. So we will encode our initial input data with key we have taken

In [18]:
test_cipher = AES.new(bytes(key), AES.MODE_ECB)
print('plaintext:', textIn[0])
print('ciphertext in files:       ', bytes(list(textOut[0])))
print('ciphertext using found key:', test_cipher.encrypt(bytes(list(textIn[0]))))

plaintext: [156 135   7 127 141 111 213 234 169 189 200 111 126 215 234 164]
ciphertext in files:        b'\xb3e<\x05i\xec\xb1\x8a\x83\xc8\x91\n\xcax%R'
ciphertext using found key: b'\xb3e<\x05i\xec\xb1\x8a\x83\xc8\x91\n\xcax%R'
